In [ ]:
import numpy as np
import xarray as xr

import datetime as dt
import pandas as pd

import matplotlib.pyplot as plt

import altair as alt
alt.data_transformers.enable('json')

from sublimpy import utils
import glob
import pytz
from scipy.signal import welch, csd
from scipy.stats import chi2

# Calculate spectra using Fourier Transform

## Open raw data

In [ ]:
file_list = glob.glob("/Users/elischwat/Development/data/sublimationofsnow/sosqc_fast/*.nc")
# file_list = [ f for f in file_list if '_20230113' in f]
file_list = [ f for f in file_list if '_20230404' in f]

# file_list = [ f for f in file_list if '_20221224' in f]

# file_list = [ f for f in file_list if '_20230224' in f]
# file_list = [ f for f in file_list if '_20230313' in f]
file_list = sorted(file_list)[16:24]
file_list

In [ ]:
index_vars = ['base_time']
value_vars = [
        'u_2m_c',	'v_2m_c',	'w_2m_c',	'h2o_2m_c', 'tc_2m_c',
        'u_3m_c',	'v_3m_c',	'w_3m_c',	'h2o_3m_c', 'tc_3m_c',
        'u_5m_c',	'v_5m_c',	'w_5m_c',	'h2o_5m_c', 'tc_5m_c',
        'u_10m_c',	'v_10m_c',	'w_10m_c',	'h2o_10m_c', 'tc_10m_c',
        'u_15m_c',	'v_15m_c',	'w_15m_c',	'h2o_15m_c', 'tc_15m_c',
        'u_20m_c',	'v_20m_c',	'w_20m_c',	'h2o_20m_c', 'tc_20m_c',

        'u_3m_uw',	'v_3m_uw',	'w_3m_uw',	'h2o_3m_uw', 'tc_3m_uw',
        'u_10m_uw',	'v_10m_uw',	'w_10m_uw',	'h2o_10m_uw', 'tc_10m_uw',

        'u_3m_ue',	'v_3m_ue',	'w_3m_ue',	'h2o_3m_ue', 'tc_3m_ue',
        'u_10m_ue',	'v_10m_ue',	'w_10m_ue',	'h2o_10m_ue', 'tc_10m_ue',

        'u_3m_d',	'v_3m_d',	'w_3m_d',	'h2o_3m_d', 'tc_3m_d',
        'u_10m_d',	'v_10m_d',	'w_10m_d',	'h2o_10m_d', 'tc_10m_d',
    ]
VARIABLES = index_vars + value_vars

In [ ]:
ds = xr.open_mfdataset(
    file_list, concat_dim="time", 
    combine="nested", 
    data_vars=VARIABLES
)

In [ ]:
df = ds[VARIABLES].to_dataframe()

## Create timestamp
To use the datam, its necessary to combine 3 columns of data from the dataset to get the full timestamp. This is demonstrated below. The 'time' column actually only incudes the second and minute information. For all datapoints, the hour according to the 'time' column is 1.  The 'base_time' column indicates the hour of the day. The 'sample' column indicates the 20hz sample number. 

We demonstrate this in the plots below

In [ ]:
df = df.reset_index()

In [ ]:
df['time'] = df.apply(lambda row: dt.datetime(
        year = row['time'].year,
        month = row['time'].month,
        day = row['time'].day,
        hour = row['base_time'].hour,
        minute = row['time'].minute,
        second = row['time'].second,
        microsecond = int(row['sample'] * (1e6/20))
    ),
    axis = 1
)

In [ ]:
df = utils.modify_df_timezone(df, pytz.UTC, "US/Mountain")

## Interpolate nans

In [ ]:
for var in value_vars:
    nans_b4_interp = df[var].isna().sum()
    df[var] = df[var].interpolate()
    nans_after_interp = df[var].isna().sum()
    print(var, len(df), nans_b4_interp, nans_after_interp)

## Calculate spectra of u'u', v'v', w'w'

In [ ]:
spectrum_ls = []
for height in [2,3,5,10,20]:
    for var in ['u', 'v', 'w']:
        spectrum = pd.DataFrame(dict(zip(
            ['frequency', 'power spectrum'],
            list(welch(
                    df[f"{var}_{height}m_c"],
                    fs=20, #Hz
                    window='hann', #'hann' is the default,
                    nperseg=72000 # one hour window
            ))
        )))
        spectrum = spectrum.assign(height = height)
        spectrum = spectrum.assign(variance = f"{var}'{var}'")
        spectrum_ls.append(spectrum)
variance_spectrum_df = pd.concat(spectrum_ls)

In [ ]:
length_of_study_period_in_seconds = (df.time.max() - df.time.min()).seconds
length_of_window_in_seconds = 72000/20
n_windows_in_study_period = length_of_study_period_in_seconds/length_of_window_in_seconds
edof = 2*n_windows_in_study_period
# Calculate confidence interval
# what are these?
conf_x = 1

conf_y0 = 1
# Degrees of freedom = 2 DOF per window, multiplied by number of windows
conf = conf_y0 * edof / chi2.ppf([0.025, 0.975], edof).reshape((2,1))

uncertainty_chart = alt.Chart().transform_calculate(
    high = f"{conf_y0 + conf[0][0]}",
    low = f"{conf_y0 - conf[1][0]}",
    x = f"{conf_x}"
).mark_line(color='black').encode(
    alt.X("x:Q").title(""),
    alt.Y("low:Q").title(""),
    alt.Y2("high:Q")
)

uncertainty_chart_dot = alt.Chart().transform_calculate(
    middle = f"{conf_y0}",
    x = f"{conf_x}"
).mark_circle(color='black').encode(
    alt.X("x:Q").title(""),
    alt.Y("middle:Q").title(""),
)

In [ ]:


# Create a line with slope -5/3 (in log log space) that fits the data
fit_chart = alt.Chart(pd.DataFrame({
        'x': np.arange(0.01, 10),
        'y': 0.01*np.arange(0.01, 10)**(-5/3)
})).mark_line(color='black', strokeDash=[4,2]).encode(
    alt.X('x:Q').scale(type='log'),
    alt.Y('y:Q').scale(type='log'),
)

spectra_chart = alt.Chart().mark_line().encode(
    alt.X("frequency:Q").scale(type='log'),
    alt.Y("power spectrum:Q").scale(type='log'),
    alt.Color("height:N"),
)

alt.layer(
    spectra_chart,
    fit_chart,
    uncertainty_chart,
    uncertainty_chart_dot,
    data=variance_spectrum_df.query("frequency > 0")
).properties(
    width=200, 
    height=150
).facet(
    'variance:O'
).configure_axis(grid=False).display(renderer='svg')

In [ ]:
variance_spectrum_df.query("frequency > 0").to_csv(str(df.time.dt.date.iloc[0]) + '-spectra.csv')

## Compare w'w' spectra from different months

In [ ]:
df_multipledays = pd.concat([
    pd.read_csv("2022-12-24-spectra.csv").assign(date = '2022-12-24'),
    pd.read_csv("2023-01-13-spectra.csv").assign(date = '2023-01-13'),
    pd.read_csv("2023-02-24-spectra.csv").assign(date = '2023-02-24'),
    pd.read_csv("2023-03-13-spectra.csv").assign(date = '2023-03-13')
])

In [ ]:
# alt.Chart(
#     df_multipledays[df_multipledays.variance == "w'w'"].query("height < 10")
# ).mark_line().encode(
#     alt.X("frequency:Q").scale(type='log'),
#     alt.Y("power spectrum:Q").scale(type='log'),
#     alt.Color("height:N"),
#     alt.Column("date:O"),
# ).properties(width = 300, height = 200)

In [ ]:
bin_avg_spectra_df_ls = []
for height in df_multipledays.height.unique():
    for date in df_multipledays.date.unique():
        src = df_multipledays[df_multipledays.variance == "w'w'"].query(f"height == {height}").query(f"date == '{date}'")
        src['frequency_binned'] = pd.cut(
            src['frequency'],
            np.logspace(-5, 1, 100)
        )
        bin_avg_spectra = src.groupby("frequency_binned")[['power spectrum']].mean().reset_index()
        bin_avg_spectra['frequency'] = bin_avg_spectra['frequency_binned'].apply(lambda int: (int.left + int.right)/2)
        bin_avg_spectra['height'] = height
        bin_avg_spectra['date'] = date
        bin_avg_spectra_df_ls.append(bin_avg_spectra)
bin_avg_spectra_df = pd.concat(bin_avg_spectra_df_ls)

In [ ]:
src = bin_avg_spectra_df[bin_avg_spectra_df.date.isin(['2022-12-24', '2023-03-13'])].query("height < 10")
src['date'] = src['date'].replace({
    '2022-12-24': 'Dec. 24\n(7cm of snow)',
    '2023-03-13': 'Mar. 13\n(126cm of snow)',
})
power_spectra_2m_is_too_low_chart = (
    uncertainty_chart
    +
    uncertainty_chart_dot
    +
    alt.Chart(
        src[['power spectrum', 'frequency', 'height', 'date']]
    ).mark_line().encode(
        alt.X("frequency:Q").scale(domain = [0.001, 10], type='log').title("Frequency (Hz)"),
        alt.Y("power spectrum:Q").scale(type='log').title("Cospectra(w'q')"),
        alt.Color("height:N"),
        alt.StrokeDash("date:O"),
).properties(width = 200, height = 200)
).configure_axis(grid=False).configure_legend(orient='bottom-left', columns = 1
)
power_spectra_2m_is_too_low_chart

In [ ]:
power_spectra_2m_is_too_low_chart.save("../../figures/power_spectra_2m_is_too_low_chart.png", ppi=200)

In [ ]:
src = bin_avg_spectra_df[bin_avg_spectra_df.date.isin(['2022-12-24', '2023-03-13'])]
src['date'] = src['date'].replace({
    '2022-12-24': '2022-12-24 (7cm snowdepth)',
    '2023-03-13': '2023-03-13 (126cm snowdepth)',
})
alt.Chart(
    src
).mark_line().encode(
    alt.X("frequency:Q").scale(type='log'),
    alt.Y("power spectrum:Q").scale(type='log'),
    alt.Color("height:N"),
    alt.Column("date:O"),
    # alt.StrokeDash("date:O"),
).properties(width = 300, height = 200).configure_axis(grid=False)

## Calculate cospectra of u'w', w'tc', and w'h2o'

In [ ]:
local_df_list = []
towers = ['c', 'd', 'uw', 'ue']
for tower in towers:
    if tower == 'c':
        heights = [2, 3, 5, 10, 15, 20]
    else:
        heights = [3, 10]
    for height in heights:
        local_df = pd.DataFrame(dict(zip(
            ['frequency', 'power spectrum'],
            list(csd(
                    np.sqrt(
                        df[f'u_{height}m_{tower}']**2 + df[f'v_{height}m_{tower}']**2
                    ),
                    df[f'w_{height}m_{tower}'],
                    fs=20, #Hz
                    window='hann', #'hann' is the default,
                    nperseg=72000
            ))
        ))).assign(height=height).assign(tower=tower)
        # local_df['power spectrum'] = np.real(local_df['power spectrum'])
        local_df['cospectrum'] = local_df['power spectrum'].apply(lambda complex: complex.real)
        local_df['quadrature spectrum'] = local_df['power spectrum'].apply(lambda complex: complex.imag)
        local_df_list.append(local_df)
momentum_copower_spectrum = pd.concat(local_df_list)

In [ ]:
local_df_list = []
towers = ['c', 'd', 'uw', 'ue']
for tower in towers:
    if tower == 'c':
        heights = [2, 3, 5, 10, 15, 20]
    else:
        heights = [3, 10]
    for height in heights:
        local_df = pd.DataFrame(dict(zip(
            ['frequency', 'power spectrum'],
            list(csd(
                    df[f'w_{height}m_{tower}'],
                    df[f'tc_{height}m_{tower}'],
                    fs=20, #Hz
                    window='hann', #'hann' is the default,
                    nperseg=72000
            ))
        ))).assign(height=height).assign(tower=tower)
        local_df['cospectrum'] = local_df['power spectrum'].apply(lambda complex: complex.real)
        local_df['quadrature spectrum'] = local_df['power spectrum'].apply(lambda complex: complex.imag)
        local_df_list.append(local_df)
sensheat_copower_spectrum = pd.concat(local_df_list)

In [ ]:
local_df_list = []
towers = ['c', 'd', 'uw', 'ue']
for tower in towers:
    if tower == 'c':
        heights = [2, 3, 5, 10, 15, 20]
    else:
        heights = [3, 10]
    for height in heights:
        local_df = pd.DataFrame(dict(zip(
            ['frequency', 'power spectrum'],
            list(csd(
                    df[f'w_{height}m_{tower}'],
                    df[f'h2o_{height}m_{tower}'],
                    fs=20, #Hz
                    window='hann', #'hann' is the default,
                    nperseg=72000
            ))
        ))).assign(height=height).assign(tower=tower)
        local_df['cospectrum'] = local_df['power spectrum'].apply(lambda complex: complex.real)
        local_df['quadrature spectrum'] = local_df['power spectrum'].apply(lambda complex: complex.imag)
        local_df_list.append(local_df)
latheat_copower_spectrum = pd.concat(local_df_list)

In [ ]:
alt.Chart(
    momentum_copower_spectrum.query("tower == 'c'").query("frequency > 0")
).mark_line().encode(
    alt.X("frequency:Q").scale(type='log'),
    alt.Y("cospectrum:Q"),
    alt.Color("height:N")
).properties(width=300, height=150, title="u'w'") |\
alt.Chart(
    sensheat_copower_spectrum.query("tower == 'c'").query("frequency > 0")
).mark_line().encode(
    alt.X("frequency:Q").scale(type='log'),
    alt.Y("cospectrum:Q"),
    alt.Color("height:N")
).properties(width=300, height=150, title="w'tc'")  |\
alt.Chart(
    latheat_copower_spectrum.query("tower == 'c'").query("frequency > 0")
).mark_line().encode(
    alt.X("frequency:Q").scale(type='log'),
    alt.Y("cospectrum:Q"),
    alt.Color("height:N")
).properties(width=300, height=150, title="w'h2o'")

In [ ]:
latheat_copower_spectrum.query("tower == 'c'").query("frequency > 0").query(
        "frequency < 0.01"
    )

In [ ]:
alt.Chart(
    latheat_copower_spectrum.query("tower == 'c'").query("frequency > 0").query(
        "frequency < 0.01"
    )
).mark_line().encode(
    alt.X("frequency:Q").scale(type='log'),
    alt.Y("cospectrum:Q").scale(type='symlog'),
    alt.Color("height:N")
).properties(width=300, height=150, title="w'h2o'")

In [ ]:
alt.Chart(
    latheat_copower_spectrum.query("tower == 'c'").query("frequency > 0").query("height < 10")
).mark_line().encode(
    alt.X("frequency:Q").scale(type='log'),
    alt.Y("power spectrum:Q"),
    alt.Color("height:N")
).properties(width=300, height=150, title="w'h2o'")

In [ ]:
alt.Chart(
    sensheat_copower_spectrum[
        sensheat_copower_spectrum.height.isin([3,10])
    ].query("frequency > 0")
).mark_line().encode(
    alt.X("frequency:Q").scale(type='log'),
    alt.Y("power spectrum:Q"),
    alt.Color("height:N"),
    alt.Facet("tower:N", columns=2),
).properties(width=200, height=100, title="w'tc'").display(renderer='svg')

# Multiresolution decomposition

Following Michi Haugeneders PHD Thesis

In [ ]:
start_date = '20221101'
end_date = '20230619'
# open files
# tidy_df = pd.read_parquet(f'tidy_df_{start_date}_{end_date}_noplanar_fit_clean.parquet')
tidy_df = pd.read_parquet(f'tidy_df_{start_date}_{end_date}_planar_fit_multiplane.parquet')
# convert time column to datetime
tidy_df['time'] = pd.to_datetime(tidy_df['time'])
tidy_df = utils.modify_df_timezone(tidy_df, pytz.UTC, 'US/Mountain')
# limit data to our dates of interest, based on continuous snow cover at Kettle Ponds
tidy_df = tidy_df.set_index('time').sort_index().loc[
    '20221130':'20230508'
].reset_index()

In [ ]:

# Identify lists of timestamps for different categories
bs_times = set(
    tidy_df.query("variable == 'SF_avg_1m_ue'").query("value > 0").time
).union(
    set(tidy_df.query("variable == 'SF_avg_2m_ue'").query("value > 0").time)
)
nobs_times = set(tidy_df.time).difference(bs_times)

ri_stable_times = tidy_df.query("variable == 'Ri_3m_c'").query("value > 0.25").time
ri_unstable_times = tidy_df.query("variable == 'Ri_3m_c'").query("value < -0.01").time
ri_neutral_times = tidy_df.query("variable == 'Ri_3m_c'").query("value >= -0.01").query("value <= 0.25").time

tgrad_stable_times = tidy_df.query("variable == 'temp_gradient_3m_c'").query("value > 0.01").time
tgrad_unstable_times = tidy_df.query("variable == 'temp_gradient_3m_c'").query("value < -0.01").time
tgrad_neutral_times = tidy_df.query("variable == 'temp_gradient_3m_c'").query("value >= -0.01").query("value <= 0.01").time

In [ ]:
src = tidy_df.query("measurement == 'potential virtual temperature'")
src = src[src.time > '20230404 1000'][src.time < '20230404 1700']
alt.Chart(src).mark_line().encode(
    alt.X('mean(value):Q').sort('-y'),
    alt.Y('height:Q'),
    alt.Facet('hours(time):T', columns=4)
).properties(width=150, height = 150)

## Calculate here (w/ Michi's/Ivana's implementation)

### Some brief examples

In [ ]:
import numpy as np
"""
    mrd(data_a::Vector, data_b::Vector, M::Integer, Mx::Integer)

Multiresolution Flux Decomposition. Adapted from Ivana Stiperski's code.
See Vickers&Mahrt 2003 'The Cospectral Gap and Turbulent Flux Calculations'
"""
def mrd(data_a, data_b, M, Mx):
    D = np.zeros(M - Mx)
    data_a2 = data_a.copy()
    data_b2 = data_b.copy()
    for ims in range(M - Mx + 1):
        ms = M - ims
        l = 2 ** ms
        nw = round((2 ** M) / l)
        sumab = 0.0
        for i in range(nw):
            k = i * l #startidx of averaging segment
            za = data_a2[k]
            zb = data_b2[k]
            for j in range(k + 1, k + l): #iterate over averaging segment
                za += data_a2[j]
                zb += data_b2[j]
            za /= l
            zb /= l
            sumab += za * zb
            for j in range(k, i * l + 1): #substract mean for next step
                data_a2[j] -= za
                data_b2[j] -= zb
        if nw > 1:
            D[ms] = sumab / nw
    return D

def newmrd(data_a, data_b, M, Mx):
    D = np.zeros(M - Mx)
    Dstd = np.copy(D)
    data_a2 = np.copy(data_a)
    data_b2 = np.copy(data_b)
    for ims in range(M - Mx + 1):
        ms = M - ims
        l = 2 ** ms
        nw = round((2 ** M) / l)
        wmeans_a = np.zeros(nw)
        wmeans_b = np.copy(wmeans_a)
        for i in range(nw):
            k = round(i * l)
            wmeans_a[i] = np.mean(data_a2[k:(i+1)*l])
            wmeans_b[i] = np.mean(data_b2[k:(i+1)*l])
            data_a2[k:(i+1)*l] -= wmeans_a[i]
            data_b2[k:(i+1)*l] -= wmeans_b[i]
        if nw > 1:
            D[ms] = np.mean(wmeans_a * wmeans_b)
            Dstd[ms] = np.std(wmeans_a * wmeans_b, ddof=0)
    return D, Dstd

In [ ]:
M = 19

mrd_src = df[['time','u_5m_c', 'v_5m_c', 'w_5m_c', 'tc_5m_c']]
mrd_src = mrd_src.rename(columns={
    'u_5m_c': 'u',
    'v_5m_c': 'v',
    'w_5m_c': 'w',
    'tc_5m_c': 'T'
})
mrd_src = mrd_src.head(2**M)
assert len(mrd_src) == 2**M

mrd_src_interpolated = mrd_src.copy()
mrd_src_interpolated['w'] = mrd_src_interpolated['w'].interpolate()
mrd_src_interpolated['T'] = mrd_src_interpolated['T'].interpolate()
result =        mrd(mrd_src_interpolated['w'], mrd_src_interpolated['T'], M, 0)
result_orth =   newmrd(mrd_src_interpolated['w'], mrd_src_interpolated['T'], M, 0)


timestep = (mrd_src_interpolated['time'].iloc[1] - mrd_src_interpolated['time'].iloc[0]).total_seconds() * 1000
mrd_x = np.array([dt.timedelta(milliseconds=2**i * timestep).total_seconds() for i in range(1, M+1)])

In [ ]:
result_df = pd.DataFrame({
    'tau':      mrd_x,
    'Co':       result,
    'Co_orth':  result_orth[0],
    'std_orth': result_orth[1]
})

(alt.Chart(result_df).mark_line().encode(
    alt.X('tau:Q').scale(type='log').title('tau (s)'),
    alt.Y('Co_orth:Q').title('C_wT (K m/s)')
) + alt.Chart(result_df).mark_errorband().transform_calculate(
    upper = 'datum.Co_orth + datum.std_orth',
    lower = 'datum.Co_orth - datum.std_orth'
).encode(
    alt.X('tau:Q').scale(type='log'),
    alt.Y('lower:Q').title('').scale(domain=[-0.005,0.005], clamp=True),
    alt.Y2('upper:Q'),
).properties(
    title = [f"{str(mrd_src_interpolated.time.dt.date.iloc[0])}, " + 
    f"0{mrd_src_interpolated.time.dt.hour.min()}00 - " +
    f"{mrd_src_interpolated.time.dt.hour.max()}00",
    '(shading shows ± 1*σ)'
    ]
).properties(width=200, height=200)).display(renderer='svg')

## Precomputed

In [ ]:
ls /Users/elischwat/Development/data/sublimationofsnow/mrd/

In [ ]:
sh_mrds = pd.read_parquet("/Users/elischwat/Development/data/sublimationofsnow/mrd/0900_1700/sensible_heat")
lh_mrds = pd.read_parquet("/Users/elischwat/Development/data/sublimationofsnow/mrd/0900_1700/latent_heat")

sh_mrds = sh_mrds[(sh_mrds['date'] >= '20221130') & (sh_mrds['date'] < '20230509')]
lh_mrds = lh_mrds[(lh_mrds['date'] >= '20221130') & (lh_mrds['date'] < '20230509')]
sh_mrds.head(3), lh_mrds.head(3)

In [ ]:
tempgrad_3m = tidy_df.query("variable == 'temp_gradient_2m_c'")
tempgrad_3m['date_str'] = tempgrad_3m.time.dt.strftime('%Y%m%d')
tempgrad_3m = tempgrad_3m[tempgrad_3m.date_str.isin(sh_mrds.date)]
tempgrad_3m = tempgrad_3m[tempgrad_3m.time.dt.hour.isin([9,10,11,12,13,14,15,16,17])]
tempgrad_3m = tempgrad_3m.groupby('date_str').value.mean()
alt.Chart(pd.DataFrame(tempgrad_3m)).mark_bar().encode(
    alt.X('value:Q').bin(True), alt.Y('count():Q')
)

In [ ]:
sh_mrds = sh_mrds.merge(
    tempgrad_3m,
    how='left',
    left_on='date',
    right_on='date_str'
)
lh_mrds = lh_mrds.merge(
    tempgrad_3m,
    how='left',
    left_on='date',
    right_on='date_str'
)

## Composite analysis of Cospectra

In [ ]:
alt.Chart(
    sh_mrds[sh_mrds.date > '20221130'][sh_mrds.height.isin([3,10, 20])]
).mark_line(opacity=0.25).encode(
    alt.X('tau:Q').scale(type='log'),
    alt.Y('Co:Q').scale(domain = [-0.02, 0.02], clamp=True),
    alt.Color('value:Q').scale(scheme='purpleorange', domain = [-0.8, 0.8]),
    alt.Detail('date:O'),
    alt.Column('tower:N'),
    alt.Row('height:O')
).configure_axis(grid=False).properties(height = 200)

In [ ]:
def q1(x):
    return x.quantile(0.25)

def q3(x):
    return x.quantile(0.75)

In [ ]:
line = alt.Chart().transform_calculate(y = '0').mark_rule().encode(y='y:Q')

src = sh_mrds[sh_mrds.height.isin([3,5, 10, 15, 20])].query("tower == 'c'")
src['stability'] = src.value.apply(lambda delta_t: 'stable' if delta_t > 0 else 'unstable')
mean_src = pd.DataFrame(src.groupby(['tau', 'height', 'tower', 'stability'])['Co'].mean())
stddev_src = pd.DataFrame(src.groupby(['tau', 'height', 'tower', 'stability'])['Co'].std())
quantiles_src = pd.DataFrame(src.groupby(['tau', 'height', 'tower', 'stability'])['Co'].agg([q1, q3]))
mean_src = mean_src.rename(columns={'Co': 'Co mean'})
stddev_src = stddev_src.rename(columns={'Co': 'Co std'})
src = mean_src.join(stddev_src).join(quantiles_src).reset_index()

src['Co lower'] = src['Co mean'] - src['Co std']
src['Co upper'] = src['Co mean'] + src['Co std']

shflux_chart = alt.layer(
    alt.Chart().mark_rule(strokeDash=[2,2], color='red').transform_calculate(
        x = '1800'
    ).encode(alt.X('x:Q').title('')),
    alt.Chart().mark_errorband().encode(
        alt.X('tau:Q').scale(type='log').title('𝜏 (s)'),
        alt.Y('q1:Q').scale(domain=[-0.0025,0.0025], clamp=True).title("Co(w'T')"),
        alt.Y2('q3:Q'),
        alt.Color('stability:N')
    ),
    alt.Chart().mark_line().encode(
        alt.X('tau:Q').scale(type='log'),
        alt.Y('Co mean:Q').scale(domain=[-0.0025,0.0025], clamp=True).title("Co(w'T')"),
        alt.Color('stability:N')
    ),
    line,
    data = src
).properties(height = 100, width = 200).facet(
    'height:O'
).properties(title='Sensible heat flux, multiresolution decomposition')

In [ ]:
line = alt.Chart().transform_calculate(y = '0').mark_rule().encode(y='y:Q')

src = lh_mrds[lh_mrds.height.isin([3,5, 10, 15, 20])].query("tower == 'c'")
src['stability'] = src.value.apply(lambda delta_t: 'stable' if delta_t > 0 else 'unstable')
mean_src = pd.DataFrame(src.groupby(['tau', 'height', 'tower', 'stability'])['Co'].mean())
stddev_src = pd.DataFrame(src.groupby(['tau', 'height', 'tower', 'stability'])['Co'].std())
quantiles_src = pd.DataFrame(src.groupby(['tau', 'height', 'tower', 'stability'])['Co'].agg([q1, q3]))
mean_src = mean_src.rename(columns={'Co': 'Co mean'})
stddev_src = stddev_src.rename(columns={'Co': 'Co std'})
src = mean_src.join(stddev_src).join(quantiles_src).reset_index()

src['Co lower'] = src['Co mean'] - src['Co std']
src['Co upper'] = src['Co mean'] + src['Co std']

lhflux_chart = alt.layer(
    alt.Chart().mark_rule(strokeDash=[2,2], color='red').transform_calculate(
        x = '1800'
    ).encode(alt.X('x:Q').title('')),
    alt.Chart().mark_errorband().encode(
        alt.X('tau:Q').scale(type='log').title('𝜏 (s)'),
        alt.Y('q1:Q').scale(domain=[-0.0025,0.0025], clamp=True).title("Co(w'q')"),
        alt.Y2('q3:Q'),
        alt.Color('stability:N')
    ),
    alt.Chart().mark_line().encode(
        alt.X('tau:Q').scale(type='log'),
        alt.Y('Co mean:Q').scale(domain=[-0.0025,0.0025], clamp=True).title("Co(w'q')"),
        alt.Color('stability:N')
    ),
    line,
    data = src
).properties(height = 100, width = 200).facet(
    'height:O', 
).properties(title='Lensible heat flux, multiresolution decomposition')

In [ ]:
(shflux_chart & lhflux_chart).display(renderer='svg')

## Calculate Ogives

In [ ]:
lh_ogives = pd.DataFrame()
for keys,df in lh_mrds[['date', 'height', 'tower', 'tau', 'Co']].groupby(['date', 'height', 'tower']).__iter__():
    df['cumsum'] = df['Co'].cumsum()
    lh_ogives = pd.concat([lh_ogives, df])
# lh_ogives = pd.DataFrame(lh_mrds.groupby(['date', 'height', 'tower']).transform(lambda df: df.set_index('tau')['Co'].cumsum()))
lh_ogives

In [ ]:
lh_ogives = lh_ogives.merge(
    tempgrad_3m,
    how='left',
    left_on='date',
    right_on='date_str'
)
lh_ogives['stability'] = lh_ogives.value.apply(lambda delta_t: 'stable' if delta_t > 0 else 'unstable')
lh_ogives

## Composite analysis of Ogives

In [21]:
rule_30min = alt.Chart().mark_rule(strokeDash=[2,2], color='red').transform_calculate(
        x = '1800'
    ).encode(alt.X('x:Q').title(''))
alt.layer(
    alt.Chart().mark_line(point=True).encode(
        alt.X('tau:Q').scale(type='log').title('𝜏 (s)'),
        alt.Y('mean(cumsum):Q').title("og(w'q')"),
        alt.Color('height:O').scale(scheme='turbo'),
    ),
    rule_30min,
    data = lh_ogives.query("tower == 'c'")
).properties(width=200, height = 200).facet('stability:N').display(renderer='svg')
# .transform_filter(
#         # alt.FieldOneOfPredicate('height', [3,10,20])
# )

alt.FacetChart(...)

In [ ]:
alt.Chart(
    lh_ogives[lh_ogives.date > '20221130'][lh_ogives.height.isin([3,10, 20])]
).mark_line(point=True).encode(
    alt.X('tau:Q').scale(type='log'),
    alt.Y('mean(cumsum):Q'),
    # alt.Detail('date:O'),
    alt.Color('height'),
    alt.Column('tower:N'),
).configure_axis(grid=False).properties(height = 200)

In [ ]:
alt.Chart(
    lh_ogives[lh_ogives.date > '20221130'][lh_ogives.height.isin([3,10, 20])]
).mark_line(point=True).encode(
    alt.X('tau:Q').scale(type='log'),
    alt.Y('mean(cumsum):Q'),
    alt.Color('stability:N'),
    alt.Column('tower:N'),
    alt.Row('height:O')
).configure_axis(grid=False).properties(height = 200)

In [ ]:
alt.Chart(
    lh_ogives[lh_ogives.date > '20221130'][lh_ogives.height.isin([3,10, 20])]
).mark_line(point=True).encode(
    alt.X('tau:Q').scale(type='log'),
    alt.Y('mean(cumsum):Q'),
    alt.Row('stability:N'),
    alt.Column('tower:N'),
    alt.Color('height:N')
).configure_axis(grid=False).properties(height = 200)

## Case study analysis of Cospectra and Ogives

In [22]:
(alt.Chart(lh_mrds.query("date == '20230417'")).mark_line().encode(
    alt.X('tau:Q').scale(type='log'),
    alt.Y('Co:Q').title("co(w'q')"),
    alt.Color('height:N'),
    alt.Column('tower:N'),
).properties(width=150, height=150) &
alt.Chart(lh_ogives.query("date == '20230417'")).mark_line().encode(
    alt.X('tau:Q').scale(type='log'),
    alt.Y('cumsum:Q').title("og(w'q')"),
    alt.Color('height:N'),
    alt.Column('tower:N'),
).properties(width=150, height=150)
).properties(title='April 17, 2023 0900-1700').display(renderer='svg')

alt.VConcatChart(...)

In [23]:
(alt.Chart(lh_mrds.query("date == '20230505'")).mark_line().encode(
    alt.X('tau:Q').scale(type='log'),
    alt.Y('Co:Q').title("co(w'q')"),
    alt.Color('height:N'),
    alt.Column('tower:N'),
).properties(width=150, height=150) &
alt.Chart(lh_ogives.query("date == '20230505'")).mark_line().encode(
    alt.X('tau:Q').scale(type='log'),
    alt.Y('cumsum:Q').title("og(w'q')"),
    alt.Color('height:N'),
    alt.Column('tower:N'),
).properties(width=150, height=150)
).properties(title='May 5, 2023 0900-1700').display(renderer='svg')

alt.VConcatChart(...)

In [26]:
co_and_og_20230417 = (   
    alt.Chart(
        lh_mrds[lh_mrds.height.isin([3,10,20])].query("tower == 'c'").query("date == '20230417'").query("tau <= 14000")
    ).mark_line().encode(
        alt.X('tau:Q').scale(type='log').title('𝜏 (s)'),
        alt.Y('Co:Q').title("co(w'q')"),
        alt.Color('height:N'),
    ) 
    +
    alt.Chart(
        lh_ogives[lh_ogives.height.isin([3,10,20])].query("tower == 'c'").query("date == '20230417'").query("tau <= 14000")
    ).mark_line(strokeDash=[2,1]).encode(
        alt.X('tau:Q').scale(type='log'),
        alt.Y('cumsum:Q').title("og(w'q')"),
        alt.Color('height:N'),
    ) 
).resolve_scale(y='independent').properties(width=200, height = 200, title='20230417')

co_and_og_20230505 = (   
    alt.Chart(
        lh_mrds[lh_mrds.height.isin([3,10,20])].query("tower == 'c'").query("date == '20230505'").query("tau <= 14000")
    ).mark_line().encode(
        alt.X('tau:Q').scale(type='log').title('𝜏 (s)'),
        alt.Y('Co:Q').title("co(w'q')"),
        alt.Color('height:N'),
    ) 
    +
    alt.Chart(
        lh_ogives[lh_ogives.height.isin([3,10,20])].query("tower == 'c'").query("date == '20230505'").query("tau <= 14000")
    ).mark_line(strokeDash=[2,1]).encode(
        alt.X('tau:Q').scale(type='log'),
        alt.Y('cumsum:Q').title("og(w'q')"),
        alt.Color('height:N'),
    ) 
).resolve_scale(y='independent').properties(width=200, height = 200, title='20230505')

((co_and_og_20230417 | co_and_og_20230505).resolve_scale(y='shared')).display(renderer='svg')

alt.HConcatChart(...)

In [27]:
rule_30min = alt.Chart().mark_rule(strokeDash=[6,2], color='black').transform_calculate(
    x = '1800'
).encode(alt.X('x:Q').title('')) 

((rule_30min+ co_and_og_20230417) | (rule_30min+ co_and_og_20230505)).display(
    renderer='svg'
)

alt.HConcatChart(...)

In [ ]:
(alt.Chart(lh_mrds.query("date == '20230417'")).mark_line().encode(
    alt.X('tau:Q').scale(type='log'),
    alt.Y('Co:Q').title("co(w'q')"),
    alt.Color('height:N'),
    alt.Column('tower:N'),
).properties(width=150, height=150) &
alt.Chart(lh_ogives.query("date == '20230417'")).mark_line().encode(
    alt.X('tau:Q').scale(type='log'),
    alt.Y('cumsum:Q').title("og(w'q')"),
    alt.Color('height:N'),
    alt.Column('tower:N'),
).properties(width=150, height=150)
).properties(title='April 17, 2023 0900-1700').display(renderer='svg')

## Category analysis of Cospectra

In [ ]:
lhflux_diff_df = tidy_df[tidy_df.variable.isin(['w_h2o__3m_c', 'w_h2o__20m_c'])].pivot(index='time', columns='variable', values='value')
lhflux_diff_df = lhflux_diff_df[lhflux_diff_df.index.hour.isin([12,13,14,15])]
lhflux_diff_df['diff'] = lhflux_diff_df['w_h2o__20m_c'] - lhflux_diff_df['w_h2o__3m_c']
lhflux_diff_df = lhflux_diff_df.groupby(lhflux_diff_df.index.date).mean()

dates_with_significant_turbfluxdiv = (lhflux_diff_df[lhflux_diff_df['diff'] < -0.001]).index
dates_without_significant_turbfluxdiv = (lhflux_diff_df[lhflux_diff_df['diff'] >= -0.001]).index

dates_with_significant_turbfluxdiv = pd.Series(dates_with_significant_turbfluxdiv).apply(lambda date: date.strftime('%Y%m%d'))
dates_without_significant_turbfluxdiv = pd.Series(dates_without_significant_turbfluxdiv).apply(lambda date: date.strftime('%Y%m%d'))

### During negative vcertical turb. flux divergence


In [ ]:
(alt.Chart(lh_mrds[lh_mrds.date.isin(dates_with_significant_turbfluxdiv)]).mark_line().encode(
    alt.X('tau:Q').scale(type='log'),
    alt.Y('mean(Co):Q').title("co(w'q')"),
    alt.Color('height:N'),
    alt.Column('tower:N'),
).properties(width=150, height=150) &
alt.Chart(lh_ogives[lh_ogives.date.isin(dates_with_significant_turbfluxdiv)]).mark_line().encode(
    alt.X('tau:Q').scale(type='log'),
    alt.Y('mean(cumsum):Q').title("og(w'q')"),
    alt.Color('height:N'),
    alt.Column('tower:N'),
).properties(width=150, height=150)
).display(renderer='svg')

### During little vertical turb. flux divergence


In [ ]:
(alt.Chart(lh_mrds[lh_mrds.date.isin(dates_without_significant_turbfluxdiv)]).mark_line().encode(
    alt.X('tau:Q').scale(type='log'),
    alt.Y('mean(Co):Q').title("co(w'q')"),
    alt.Color('height:N'),
    alt.Column('tower:N'),
).properties(width=150, height=150) &
alt.Chart(lh_ogives[lh_ogives.date.isin(dates_without_significant_turbfluxdiv)]).mark_line().encode(
    alt.X('tau:Q').scale(type='log'),
    alt.Y('mean(cumsum):Q').title("og(w'q')"),
    alt.Color('height:N'),
    alt.Column('tower:N'),
).properties(width=150, height=150)
).display(renderer='svg')